In [1]:

instagram_crawling

NameError: name 'instagram_crawling' is not defined

In [ ]:
ss

In [15]:
import pandas as pd
instagram_crawling =pd.read_csv('C:/Users/junseok/Downloads/instagram_crawling.csv')
instagram_crawling =pd.read_csv('C:/Users/junseok/Downloads/instagram_crawling.csv')
instagram_crawling[0:20]

,id,text,hashtag
0,hellopersonv,갈비 - 아 여기 갈비 대존맛이다.. 저번에 갓던 숙대에 있던 갈비보다 백배 천배 ...,"['#맛집', '#먹스타그램', '#서울맛집', '#맛집추천', '#챙맛집', '#..."
1,hellopersonv,카페- 약수역 항상 들리는 카페!!!!! 여기 커피 너무 맛있고 분위기도 너무 좋다...,"['#맛집추천', '#챙맛집', '#먹방', '#먹스타그램', '#먹스타맞팔', '..."
2,e_ddomuck,"⠀ ⠀ ⠀ 오늘은 남성들의 원픽, 돈까스 가락동 돈까스의 거장 점심시간에도 사람 바...","['#가락동맛집', '#돈까스맛집', '#윤식이돈까스', '#경찰병원역맛집']"
3,e_ddomuck,⠀ ⠀ ⠀ 이탈리안 음식 좋아하는 또리에게 동네 파스타 맛집 물어보고 추천 받아서 ...,"['#문정동맛집', '#파스타맛집', '#피자맛집', '#라치오', '#문정로데오거..."
4,e_ddomuck,⠀ ⠀ ⠀ 식객 허영만의 돈타래간장게장 저렴한 가격에 오동통한 간장게장을 맛 볼 수...,"['#인천맛집', '#간장게장맛집', '#백반기행맛집', '#동암역맛집']"
5,e_ddomuck,⠀ ⠀ ⠀ 어제 뭐했는지는 기억 못해도 뭐 먹은지는 기억하는 나와 내친구들..😅💗 ...,"['#오늘의집밥', '#훈제오리미나리볶음']"
6,e_ddomuck,⠀ ⠀ ⠀ 윤기 좌르르한 흰밥에 싸먹는 제일 맛있는 집 김밥! 나름 밥 조금 넣는다...,"['#오늘의집밥', '#김밥']"
7,e_ddomuck,⠀ ⠀ ⠀ 아니 이게 뭐람?!??? 봄이 찾아오는줄 알았는데 갑자기 또 추워진 날씨...,"['#오늘의집밥', '#바지락칼국수']"
8,e_ddomuck,⠀ ⠀ ⠀ 예전에 포스팅 했었던 에서 달려온 싱싱한 문어🐙🐙🐙🐙 오늘 저녁은 완벽한...,"['#오늘의집밥', '#문어삼합', '#양양설악회센터', '#만성호', '#문어숙회..."
9,e_ddomuck,⠀ ⠀ ⠀ 1월부터 5월까지는 빨갛고 상큼한 딸기철🍓❤️ 딸기로 만든 여러가지 요리...,"['#광고', '#빕스', '#딸기홀릭🍓', '#빕스딸기홀릭', '#VIPS', '..."


In [1]:
from konlpy.tag import Mecab
from tqdm import tqdm
import re
import pickle
import csv
import pandas as pd
from pandas import DataFrame 
import numpy as np

In [2]:
def clean_text(text):
    text = text.replace(".", "").strip()
    text = text.replace(",", " ").strip()
    pattern = '[^ ㄱ-ㅣ가-힣|0-9]+'
    text = re.sub(pattern=pattern, repl='', string=text)
    return text

In [3]:
def get_nouns(tokenizer, sentence):
    tagged = tokenizer.pos(sentence)
    nouns = [s for s, t in tagged if t in ['NNG', 'NNP', 'VA', 'XR'] and len(s) >1]
    return nouns

def tokenize(df):
    tokenizer = Mecab(dicpath=r'C:/mecab/mecab-ko-dic')
    processed_data = []
    for sent in tqdm(df['text']):
        sentence = clean_text(str(sent).replace("\n", "").strip())
        processed_data.append(get_nouns(tokenizer, sentence))
    return processed_data
def save_processed_data(processed_data):
    with open("tokenized_data_", 'w', newline="", encoding='utf-8') as f:
        writer = csv.writer(f)
        for data in processed_data:
            writer.writerow(data)
            


In [4]:
if __name__ == '__main__':
    df = pd.read_csv("C:/Users/junseok/Downloads/instagram_crawling.csv")
    df.columns=['id', 'text', 'hashtag']
    df.dropna(how='any')
    processed_data = tokenize(df)
    save_processed_data(processed_data)

100%|██████████| 3157/3157 [00:01<00:00, 2773.51it/s]


In [5]:

from konlpy.tag import Mecab
mecab = Mecab(dicpath=r'C:/mecab/mecab-ko-dic')
print(mecab.pos('아바지가방에들어가신다'))

[('아바지', 'NNG'), ('가', 'JKS'), ('방', 'NNG'), ('에', 'JKB'), ('들어가', 'VV'), ('신다', 'EP+EC')]


In [6]:
processed_data = [sent.strip().split(",") for sent in tqdm(open("tokenized_data_",'r',encoding='utf-8').readlines())]
processed_data = DataFrame(processed_data)
processed_data[0] = processed_data[0].replace("", np.nan)
processed_data = processed_data[processed_data[0].notnull()]
processed_data = processed_data.values.tolist()
processed_data2=[]
for i in processed_data:
    i = list(filter(None, i))
    processed_data2.append(i)
processed_data = processed_data2
processed_data

100%|██████████| 3157/3157 [00:00<00:00, 210388.28it/s]


[['갈비',
  '갈비',
  '저번',
  '숙대',
  '갈비',
  '천배',
  '맛있',
  '맛있',
  '고기',
  '거미',
  '부드럽',
  '갈비',
  '맛있',
  '맛있',
  '마늘',
  '갈비',
  '마지막',
  '마늘',
  '디저트',
  '입가심',
  '최고'],
 ['카페',
  '약수역',
  '카페',
  '커피',
  '맛있',
  '분위기',
  '크림',
  '카푸치노',
  '모카',
  '크림',
  '단단',
  '커피',
  '맛있',
  '브륄',
  '레라',
  '메뉴',
  '디저트',
  '크림',
  '브륄',
  '비슷',
  '커피',
  '적용',
  '설탕',
  '커피',
  '그니',
  '맛있',
  '버트',
  '서울',
  '디저트',
  '크로',
  '플레이트',
  '맛있',
  '약수역',
  '동대입구역',
  '강추',
  '카페'],
 ['오늘',
  '남성',
  '까스',
  '가락동',
  '까스',
  '거장',
  '점심',
  '시간',
  '사람',
  '바글',
  '바글',
  '가락동',
  '유명',
  '메뉴',
  '겨자',
  '소스',
  '까스',
  '매콤',
  '소스',
  '윤식',
  '까스',
  '메뉴',
  '거장',
  '냄새',
  '튀김',
  '도톰',
  '고기',
  '한입',
  '육즙',
  '경양식',
  '일본식',
  '까스',
  '싱싱',
  '양배추',
  '별거',
  '맛있',
  '사장',
  '가락동',
  '주민',
  '까스',
  '맛집',
  '윤식',
  '까스',
  '윤식',
  '까스',
  '사랑',
  '반찬',
  '까스',
  '다음',
  '포장',
  '까스',
  '윤식',
  '까스',
  '까스',
  '윤식',
  '까스'],
 ['이탈리안',
  '음식',
  '동네',
  '파스타',
  '맛집',
  '추천',
  '문정동',
  '로데오',
  

In [7]:
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [8]:
dictionary = corpora.Dictionary(processed_data)
dictionary.filter_extremes(no_below=2, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in processed_data]

2022-03-02 16:48:48,197 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-03-02 16:48:48,306 : INFO : built Dictionary(12797 unique tokens: ['갈비', '거미', '고기', '디저트', '마늘']...) from 3056 documents (total 147465 corpus positions)
2022-03-02 16:48:48,306 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(12797 unique tokens: ['갈비', '거미', '고기', '디저트', '마늘']...) from 3056 documents (total 147465 corpus positions)", 'datetime': '2022-03-02T16:48:48.306339', 'gensim': '4.1.2', 'python': '3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'created'}
2022-03-02 16:48:48,315 : INFO : discarding 5814 tokens: [('맛있', 1865), ('거장', 1), ('바글', 1), ('라치오', 1), ('무나물', 1), ('식객', 1), ('장정식', 1), ('허영', 1), ('미나리볶음', 1), ('구단', 1)]...
2022-03-02 16:48:48,315 : INFO : keeping 6983 tokens which were in no less than 2 and no more than 1528 (=50.0%) documents
2022-03-02 16:48:48,321 : INFO : resulting dictiona

In [9]:
class gensim.models.ldamodel.LdaModel(corpus=None, num_topics=100, id2word=None, distributed=False, chunksize=2000, passes=1, update_every=1, alpha='symmetric', eta=None, decay=0.5, offset=1.0, eval_every=10, iterations=50, gamma_threshold=0.001, minimum_probability=0.01, random_state=None, ns_conf=None, minimum_phi_value=0.01, per_word_topics=False, callbacks=None, dtype=<class 'numpy.float32'>)

SyntaxError: invalid syntax (Temp/ipykernel_5604/646307694.py, line 1)

In [10]:
num_topics = 5
chunksize = 2000
passes = 20
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)


2022-03-02 16:48:50,552 : INFO : using autotuned alpha, starting with [0.2, 0.2, 0.2, 0.2, 0.2]
2022-03-02 16:48:50,554 : INFO : using serial LDA version on this node
2022-03-02 16:48:50,556 : INFO : running online (multi-pass) LDA training, 5 topics, 20 passes over the supplied corpus of 3056 documents, updating model once every 2000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2022-03-02 16:48:50,557 : INFO : PROGRESS: pass 0, at document #2000/3056
2022-03-02 16:48:52,469 : INFO : optimized alpha [0.09177679, 0.06693296, 0.084355235, 0.10468205, 0.06966993]
2022-03-02 16:48:52,471 : INFO : merging changes from 2000 documents into a model of 3056 documents
2022-03-02 16:48:52,473 : INFO : topic #0 (0.092): 0.012*"고기" + 0.009*"방문" + 0.007*"서울" + 0.007*"메뉴" + 0.006*"맛집" + 0.006*"치즈" + 0.005*"가격" + 0.005*"생각" + 0.005*"소스" + 0.005*"느낌"
2022-03-02 16:48:52,474 : INFO : topic #1 (0.067): 0.016*"라멘" + 0.008*"메뉴" + 0.008*"까스" + 0

In [11]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2022-03-02 16:49:09,240 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2022-03-02 16:49:09,248 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2022-03-02 16:49:09,256 : INFO : CorpusAccumulator accumulated stats from 3000 documents


Average topic coherence: -2.4362.
[([(0.013128022, '메뉴'),
   (0.010864314, '와인'),
   (0.010063171, '음식'),
   (0.008689827, '추천'),
   (0.008244845, '방문'),
   (0.0080069285, '소스'),
   (0.007840341, '맛집'),
   (0.0074401572, '서울'),
   (0.006636478, '요리'),
   (0.00648698, '가격'),
   (0.006401764, '주문'),
   (0.006216115, '튀김'),
   (0.0062105134, '소바'),
   (0.005457637, '새우'),
   (0.005298578, '정도'),
   (0.0052946764, '분위기'),
   (0.0051438194, '생각'),
   (0.0046997825, '느낌'),
   (0.00451353, '식감'),
   (0.004334135, '국물')],
  -1.766147449710611),
 ([(0.02418167, '카페'),
   (0.017111551, '커피'),
   (0.012114228, '공간'),
   (0.011429575, '크림'),
   (0.009946764, '디저트'),
   (0.009557103, '버거'),
   (0.008909118, '딸기'),
   (0.00875509, '서울'),
   (0.008631046, '라떼'),
   (0.0079618255, '느낌'),
   (0.007881485, '케이크'),
   (0.007392903, '메뉴'),
   (0.006587959, '생각'),
   (0.006418167, '방문'),
   (0.0063718, '추천'),
   (0.0060899267, '음료'),
   (0.005962972, '분위기'),
   (0.0050392807, '초코'),
   (0.00496981, '맛집'),


In [12]:
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib_inline\config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [13]:
lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'file_name.html')

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [14]:
pyLDAvis.display(lda_visualization, 'file_name.html')